In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

#make the depreciation warnings go away
import warnings
#I'm tired of the warnings on functions the professor asks us to use :) 
warnings.filterwarnings("ignore")

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import json
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
import sys
from subprocess import check_output
print(sys.version)

 


3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


In [4]:
a = pd.read_json('train.json')
for x, y in enumerate(a.columns):
    print(x,y)

0 giver_username_if_known
1 number_of_downvotes_of_request_at_retrieval
2 number_of_upvotes_of_request_at_retrieval
3 post_was_edited
4 request_id
5 request_number_of_comments_at_retrieval
6 request_text
7 request_text_edit_aware
8 request_title
9 requester_account_age_in_days_at_request
10 requester_account_age_in_days_at_retrieval
11 requester_days_since_first_post_on_raop_at_request
12 requester_days_since_first_post_on_raop_at_retrieval
13 requester_number_of_comments_at_request
14 requester_number_of_comments_at_retrieval
15 requester_number_of_comments_in_raop_at_request
16 requester_number_of_comments_in_raop_at_retrieval
17 requester_number_of_posts_at_request
18 requester_number_of_posts_at_retrieval
19 requester_number_of_posts_on_raop_at_request
20 requester_number_of_posts_on_raop_at_retrieval
21 requester_number_of_subreddits_at_request
22 requester_received_pizza
23 requester_subreddits_at_request
24 requester_upvotes_minus_downvotes_at_request
25 requester_upvotes_minus_

In [5]:
#Nikki's first three hypotheses: date/time will matter

import datetime
def day_time(x):
    y = ''
    if datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 10:
        y = 0
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 10 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 16:
        y = 1
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 16 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 21:
        y = 2
    else: 
        y = 3
    return y

day_values = ['morning', 'midday', 'evening', 'latenight']
    
def human_time(a):
    import datetime
    from datetime import date
    import calendar
    ### for the data in raop, return human time.  maybe the time of day matters
    a['human_readable_local_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime('%Y-%m-%d %H:%M:%S')
    a['human_readable_UTC_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request_utc']).strftime('%Y-%m-%d %H:%M:%S')
    a['weekday'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).weekday()
    a['month'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).month
    a['time_of_day'] = day_time(a)
    return a


In [6]:
b = a.apply(human_time, axis=1)

In [7]:
#### add in number of downvotes.  The test data doesn't have this, it only has these aggregate values.  Derive number
### of downvotes
### we're given 
###up + down = col_26 
###and 
### up-down = col_24
### solve algebraically,
# 2up = col_26+col_24,
# up = (col_26+col_24)/2
# (col_26+col_24)/2 + down = col_26
#down = col_26 - (col_26+col_24)/2
#down = (col_26 - col_24)/2

b['total_downvotes'] = (b['requester_upvotes_plus_downvotes_at_request'] - b['requester_upvotes_minus_downvotes_at_request'])/2


In [10]:
def preprocess(x):
    """Use a series of regex expressions to remove unwanted characters"""
    #remove non-alpha-numeric characters, replace with whitespace
    x1 = re.sub(r'[^a-zA-Z_0-9_\s]'," ", x).lower()
    #replae all numbers with a single token and a space afterwards
    x1a = re.sub(r'[0-9]+', 'number ', x1)
    #x1b = re.sub(r'[_]+', ' ', x1a)
    #even though there are words that are just '_____', f1 actuall decreases when they're removed
    #remove newlines
    x2 = re.sub(r'[\n]', " ", x1a)
    #scrub out extra spaces
    x3 = re.sub(r'\s+', ' ', x2)  #other steps might have added extra space; remove
    return x3.strip()

In [24]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
#try the Pipeline implementation from sk learn

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class WeekdayExtractor(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        shape = data_dict.shape
        return data_dict[self.key].reshape(shape[0],1)

class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features

class NBWrapper(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, NB):
        self.NB = NB

    def fit(self, x, y=None):
        self.NB.fit(x, y)
        return self

    def transform(self, posts):
        return self.NB.predict(posts)

pipeline4= Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[
        ### add in text variables
            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
           #     ('cv', CountVectorizer(analyzer='char_wb', 
           #                           #vocabulary=vocabulary,
           #                            #preprocessor=preprocess,
           #                          max_df=0.5, ngram_range=(1,2))),
                ('tfidf', TfidfVectorizer(min_df=65, 
                                          preprocessor=preprocess
                                         ))
            ])),

            
            #('get-text', Pipeline([
            #    ('selector', ItemSelector(key='request_text_edit_aware')),
            #    ('tfidf', TfidfVectorizer(min_df=50))
                #('cv', CountVectorizer(analyzer='char_wb', 
                #                       #vocabulary=vocabulary,
                #                       max_df=0.5, ngram_range=(1,3)))
            #])),
         ### adding in categorical variables
           # research different encoders
           ('get-time_of_day', Pipeline([
                ('selector', WeekdayExtractor(key='time_of_day')),
                ('one_hot', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
        
           ('get-weekday', Pipeline([
                ('selector', WeekdayExtractor(key='weekday')),
                ('lh', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
          ('get-month', Pipeline([
                ('selector', WeekdayExtractor(key='month')),
                ('2h', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
                    
        ### add numeric data
        #research better numeric extractors and coders
         ('get-requester_number_of_subreddits_at_request', Pipeline([
                ('selector', WeekdayExtractor(key='requester_number_of_subreddits_at_request')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
            
         ('get-downvotes', Pipeline([
                ('selector', WeekdayExtractor(key='total_downvotes')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))   
                
           ]))
        ]
    )),

    # Use a SVC classifier on the combined features
    #('estimator',LogisticRegression(penalty='l2',C=25))
    #('estimator', SVC(kernel='linear')) 
    
    ('estimator', DecisionTreeClassifier())
    #('estimator', RandomForestClassifier())
    #('estimator', MultinomialNB())
    #('estmator', KNeighborsClassifier())
])



In [25]:
data_set = b[['request_title','requester_number_of_subreddits_at_request', 
              'time_of_day','weekday', 'month', 'total_downvotes']]
#data_set_title = a['request_title']
data_labels = b['requester_received_pizza']


sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(data_set, data_labels):
    X_train, X_test = data_set.iloc[train_index], data_set.iloc[test_index]
    y_train, y_test = data_labels.iloc[train_index], data_labels.iloc[test_index]

pipeline4.fit(X_train, y_train)
y_pred = pipeline4.predict(X_test)
target_names = ['No pizza','Got Pizza']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.77      0.80      0.78       762
  Got Pizza       0.32      0.29      0.30       248

avg / total       0.66      0.67      0.67      1010



In [26]:
confusion_matrix(y_pred, y_test)

array([[606, 176],
       [156,  72]])